<a href="https://colab.research.google.com/github/IKolzin/kandinsky-picture-maker/blob/main/Picture_maker_(Kandinsky_2_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
#@title Initialization - run this cell to install dependencies and prepare the model (usually takes several minutes) { display-mode: "form" }

from PIL import Image
import math
def image_grid(images, max_columns = 5):
    if len(images) == 0:
      return
    width, height = images[0].size
    columns = min(len(images), max_columns)
    rows = math.ceil(len(images) / max_columns)
    grid = Image.new('RGB', size=(columns*width, rows*height))

    for i, img in enumerate(images):
        grid.paste(img, box=(i%columns*width, i//columns*height))
    return grid

def install_kandinsky():
    !pip install "git+https://github.com/ai-forever/Kandinsky-2.git"
    !pip install git+https://github.com/openai/CLIP.git
    from kandinsky2 import get_kandinsky2
    model = get_kandinsky2('cuda', task_type='text2img', model_version='2.1', use_flash_attention=False)
    return model

def run_model(prompt="a cute and adorable kitten wearing a hat",
              batch_size=1, 
              num_steps=50,
              guidance_scale=4,
              prior_cf_scale=4,
              prior_steps=5):
    print(f'''Running with prompt: {prompt}
    batch_size: {batch_size}, num_steps={num_steps} guidance_scale={guidance_scale}, prior_cf_scale={prior_cf_scale}, prior_steps={prior_steps}''')
    images = model.generate_text2img(
        prompt, 
        num_steps=num_steps,
        batch_size=batch_size, 
        guidance_scale=guidance_scale,
        h=768, w=768,
        sampler='p_sampler', 
        prior_cf_scale=prior_cf_scale,
        prior_steps=str(prior_steps)
    )
    return images

model = install_kandinsky()

In [ ]:
#@title Text2Image - run this cell to get your pictures { display-mode: "form" }

#@markdown Type what you want to see
prompt = 'an adorable kitten wearing a hat' #@param {type:"string"}
#@markdown Or paste a python list with multiple prompts here (overrides the first field)
prompts = [] #@param {type:"raw"}
#@markdown If the notebook runs out of GPU memory, try reducing batch size (and restarting)
batch_size = 2 #@param {type:"slider", min:1, max:3, step:1}
#@markdown More batches directly leads to longer runtime
num_batches = 1 #@param {type:"slider", min:1, max:100, step:1}
#@markdown More steps take more time but in theory provide better quality
num_steps = 40 #@param {type:"slider", min:1, max:100, step:1}
#@markdown For Kandinsky, these paremeters don't seem to do much
guidance_scale = 4 #@param {type:"slider", min:0.1, max:10, step:0.1}
prior_cf_scale = 4 #@param {type:"slider", min:0.1, max:10, step:0.1}
prior_steps = 4 #@param {type:"slider", min:2, max:10, step:1}

result = []
iterable = range(num_batches)
if (not isinstance(prompts, list)) or (len(prompts) == 0):
    _prompts = [prompt] * num_batches
else:
    _prompts = prompts * num_batches

for prompt in _prompts:
    result.extend(run_model(
        prompt=prompt, num_steps=num_steps, batch_size=batch_size,
        guidance_scale=guidance_scale, prior_cf_scale=prior_cf_scale, 
        prior_steps=prior_steps))

image_grid(result)

In [ ]:
#@title Run this cell to display individual images { display-mode: "form" }
for img in result:
    img.show()